In [16]:
import numpy as np
import sys
sys.path.append("../")
import abus_classification
from abus_classification import features
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

In [8]:
tumors = abus_classification.datasets.TDSCTumors(path="../data/tdsc")

In [10]:
def extract_features(volume, mask):

    whr = features.radiology.height_to_width_ratio(mask)
    h = features.radiology.entropy(volume*mask)
    ai = features.radiology.average_intensity(volume*mask)
    mc = features.radiology.margin_contrast(volume=volume, mask=mask)
    vi = features.radiology.variance_of_intensities(volume*mask)
    sh = features.radiology.sphericity(mask)
    cp = features.radiology.compactness(mask)
    # pab = features.radiology.posterior_acoustic_behavior(volume, mask)
    
    return np.array([whr, h,ai,mc,vi,sh,cp], dtype=np.float32)

In [12]:
X = []
Y = []
for vol, mask, label in tumors:
    X.append(extract_features(vol, mask))
    Y.append(label)

In [17]:
def classify_with_svm(x, y):
    
    acc = 0
    train_acc = 0
    cfm = [[0,0],
           [0,0]]

    for i in range(100):
        x_test, y_test = x[i], y[i]
        X_train, Y_train = x[:i] + x[i+1:], y[:i] + y[i+1:]
        clf = SVC()
        clf.fit(X_train, Y_train)
        P = clf.predict(X_train)
        train_acc += accuracy_score(Y_train, P)
        res = clf.predict([x_test])[0]
        if res == y_test:
            acc += 1
            
        cfm[y_test][res] += 1
            
    print(f"Train accuracy: {train_acc/100}")
    print(f"Accuracy: {acc/100}")
    print(f"{cfm[0]}\n{cfm[1]}")
    
classify_with_svm(X,Y)

Train accuracy: 0.6609090909090901
Accuracy: 0.59
[52, 6]
[35, 7]
